In [ ]:
import pandas as pd
import dask.dataframe as ds
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/content/nyc_taxi_data.csv')
df.head()

print("Describe : \n",df.describe())
print("Size",df.size)

# Check for outliers (example using the 'trip_distance' column)
# You'll likely want to adjust this based on your specific data and columns
def find_outliers_iqr(column):
    q1 = column.quantile(0.25)
    q3 = column.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = column[(column < lower_bound) | (column > upper_bound)]
    return outliers

# Get a list of numerical columns
numerical_cols = df.select_dtypes(include=['number']).columns

for column in numerical_cols:
    outliers = find_outliers_iqr(df[column])
    if len(outliers) > 0:
        print(f"Outliers in '{column}':\n", outliers)


Describe : 
           VendorID  passenger_count  trip_distance   RatecodeID  PULocationID  \
count  1000.000000      1000.000000    1000.000000  1000.000000   1000.000000   
mean      1.505000         2.527000       7.760560     2.451000     25.590000   
std       0.500225         1.120056       4.238151     1.119309     14.528488   
min       1.000000         1.000000       0.510000     1.000000      1.000000   
25%       1.000000         2.000000       4.185000     1.000000     13.000000   
50%       2.000000         3.000000       7.490000     2.000000     25.000000   
75%       2.000000         4.000000      11.550000     3.000000     38.000000   
max       2.000000         4.000000      14.990000     4.000000     50.000000   

       DOLocationID  payment_type  fare_amount        extra  mta_tax  \
count   1000.000000    1000.00000  1000.000000  1000.000000   1000.0   
mean      25.936000       1.52000    28.182210     2.440790      0.5   
std       14.027854       0.49985    12.8

In [ ]:
df.columns = [x.lower() for x in df.columns]
print(df.columns)
# Drop the 'vendor_id' column
df = df.drop('vendorid', axis=1)

# Convert the categorical column 'store_and_fwd_flag'
df['store_and_fwd_flag_encoded'] = df['store_and_fwd_flag'].map({'N': 0, 'Y': 1})

print(df.head())


Index(['vendorid', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'ratecodeid', 'store_and_fwd_flag',
       'pulocationid', 'dolocationid', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'tip_percent'],
      dtype='object')
  tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  trip_distance  \
0      1/12/2023 17:19        1/1/2023 21:59                4          12.16   
1      1/14/2023 20:27       1/17/2023 13:01                3          13.14   
2       1/9/2023 15:25       1/21/2023 13:55                1          13.58   
3       1/3/2023 14:58        1/6/2023 18:23                3           9.12   
4       1/8/2023 10:34       1/22/2023 11:31                1           4.81   

   ratecodeid store_and_fwd_flag  pulocationid  dolocationid  payment_type  \
0           2                  Y            13           

In [ ]:
# Get a list of numerical columns
numerical_cols = df.select_dtypes(include=['number']).columns

for column in numerical_cols:
    outliers = find_outliers_iqr(df[column])
    if not outliers.empty:
        print(f"Outliers in '{column}':\n{outliers.head()}")



Outliers in 'tip_percent':
19     89.334342
25     79.436804
30     72.216749
46     96.480490
48    102.147971
Name: tip_percent, dtype: float64


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

df = df.dropna()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime',
             'store_and_fwd_flag'], axis=1)

X = df.drop('fare_amount', axis=1)
y = df['fare_amount']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the train set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# You can replace LogisticRegression with other models (e.g., RandomForestRegressor) depending on your task
estimator = LinearRegression()#00)  # Increase max_iter if needed for convergence
selector = RFE(estimator, n_features_to_select=10, step=1)  # Select top 10 features
selector = selector.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[selector.support_]
print("Selected features:", selected_features)

# Create new datasets with only the selected features
X_train_selected = X_train[selected_features]
X_val_selected = X_val[selected_features]
X_test_selected = X_test[selected_features]

Selected features: Index(['passenger_count', 'trip_distance', 'ratecodeid', 'payment_type',
       'extra', 'tip_amount', 'tolls_amount', 'congestion_surcharge',
       'tip_percent', 'store_and_fwd_flag_encoded'],
      dtype='object')


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

# Create models
linear_model = LinearRegression()
rf_model = RandomForestRegressor()
xgb_model = XGBRegressor()

# Train models
linear_model.fit(X_train_selected, y_train)
rf_model.fit(X_train_selected, y_train)
xgb_model.fit(X_train_selected, y_train)

# Predict on test set
linear_pred = linear_model.predict(X_test_selected)
rf_pred = rf_model.predict(X_test_selected)
xgb_pred = xgb_model.predict(X_test_selected)

# Evaluate models using Mean Squared Error (MSE)
linear_mse = mean_squared_error(y_test, linear_pred)
rf_mse = mean_squared_error(y_test, rf_pred)
xgb_mse = mean_squared_error(y_test, xgb_pred)

# Print the MSE for each model
print("Linear Regression MSE:", linear_mse)
print("Random Forest MSE:", rf_mse)
print("XGBoost MSE:", xgb_mse)

# Determine the best model based on MSE
models = {
    "Linear Regression": linear_mse,
    "Random Forest": rf_mse,
    "XGBoost": xgb_mse
}
best_model = min(models, key=models.get)
print("Best Model based on MSE:", best_model)


Linear Regression MSE: 168.10694253587798
Random Forest MSE: 179.76495674420002
XGBoost MSE: 234.58133948732888
Best Model based on MSE: Linear Regression


In [ ]:

from sklearn.model_selection import GridSearchCV

# Define parameter grids for each model
param_grid_linear = {}  # No hyperparameters to tune for Linear Regression

param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}

# Perform GridSearchCV for each model
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=5, scoring='neg_mean_squared_error')
grid_search_rf.fit(X_train_selected, y_train)

grid_search_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, cv=5, scoring='neg_mean_squared_error')
grid_search_xgb.fit(X_train_selected, y_train)


# Print the best parameters and score for each model
print("Best parameters for Random Forest:", grid_search_rf.best_params_)
print("Best MSE for Random Forest:", -grid_search_rf.best_score_)

print("Best parameters for XGBoost:", grid_search_xgb.best_params_)
print("Best MSE for XGBoost:", -grid_search_xgb.best_score_)


# Train the models with the best hyperparameters
best_rf_model = RandomForestRegressor(**grid_search_rf.best_params_)
best_rf_model.fit(X_train_selected, y_train)

best_xgb_model = XGBRegressor(**grid_search_xgb.best_params_)
best_xgb_model.fit(X_train_selected, y_train)

# Predict on the test set using the best models
best_rf_pred = best_rf_model.predict(X_test_selected)
best_xgb_pred = best_xgb_model.predict(X_test_selected)

# Evaluate the best models
best_rf_mse = mean_squared_error(y_test, best_rf_pred)
best_xgb_mse = mean_squared_error(y_test, best_xgb_pred)

print("Best Random Forest MSE:", best_rf_mse)
print("Best XGBoost MSE:", best_xgb_mse)

# Determine the best model based on the tuned MSE
models_tuned = {
    "Linear Regression": linear_mse,  # Keep the original Linear Regression MSE
    "Random Forest (Tuned)": best_rf_mse,
    "XGBoost (Tuned)": best_xgb_mse
}
best_model_tuned = min(models_tuned, key=models_tuned.get)
print("Best Model (Tuned) based on MSE:", best_model_tuned)


Best parameters for Random Forest: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 200}
Best MSE for Random Forest: 166.29650584542256
Best parameters for XGBoost: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 100}
Best MSE for XGBoost: 164.34150971024863
Best Random Forest MSE: 175.04379859136714
Best XGBoost MSE: 167.42186701201368
Best Model (Tuned) based on MSE: XGBoost (Tuned)


In [ ]:
# Predict on the validation set using the best model
y_pred = best_xgb_model.predict(X_val_selected)

# Evaluate the best model on the test set
test_mse = mean_squared_error(y_val, y_pred)
test_r2 = r2_score(y_val, y_pred)

print("Test MSE:", test_mse)
print("Test R-squared:", test_r2)

# Export the best model using pickle
import pickle

filename = 'xgb_model.pkl'
pickle.dump(best_xgb_model, open(filename, 'wb'))

print(f"Best model exported to {filename}")


Test MSE: 172.0716014779335
Test R-squared: -0.0052057184277505275
Best model exported to xgb_model.pkl


In [ ]:
# prompt: export the X_val as json file

import json

# Assuming X_val is your DataFrame
X_val_json = X_val.to_json(orient='records')

with open('X_val.json', 'w') as f:
  f.write(X_val_json)

from google.colab import files
files.download('X_val.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>